# Data Cleaning

In [1]:
#!pip install unidecode

In [2]:
#!pip install nltk

In [3]:
#!pip install datetime

In [1]:
import pandas as pd
import string
import numpy as np
from nltk.tokenize import word_tokenize
from datetime import datetime
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import unidecode
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
import os

## instalacion NLTK

In [2]:
#!pip install nltk

## instalacion datetime

In [3]:
#!pip install datetime

In [4]:
pd.set_option('max_colwidth', 800) 
data = pd.read_csv("../QueEstaPasando/data/total.csv", index_col=0)
data.head()

,Unnamed: 0.1,text,created_at,author_id,id,neg,neu,pos,score
0,0,@MercantilBanco Buenas tardes intento abrir mi mercantil en línea y dice datos y clave incorrecta...está fallando la plataforma?,2021-04-28T22:47:36.000Z,202148873,1387539021983850497,0.994753,0.004321,0.000926,neg
1,1,@MercantilBanco Buenas tardes! En esta oportunidad les escribo para saber si hay problemas con la plataforma para realizar transferencias a otras cuentas mercantil ya que al momento de realizar una transferencia me arroja el mensaje de que intente más tardes!,2021-04-28T22:46:07.000Z,1274553941917863936,1387538646832648197,0.666402,0.328037,0.005561,neg
2,2,@MercantilBanco Deseo es reponer mi tarjeta de débito,2021-04-28T22:32:15.000Z,343710274,1387535156626526209,0.107078,0.886624,0.006298,neu
3,3,"@jesusmdbtw2012 @MercantilBanco Si lo está, estoy en la misma situación",2021-04-28T22:06:42.000Z,1323258072534552577,1387528726712311812,0.003918,0.989461,0.006621,neu
4,4,"El @MercantilBanco me quito 180.000 Bs por 4 mensajes de texto al 24024 solicitando clave temporal para un pago móvil, la cual nunca llego, son ladrones o no? https://t.co/bVVmvxfhjH",2021-04-28T21:48:28.000Z,1099360339207049218,1387524137904754689,0.996315,0.002650,0.001035,neg


In [10]:
data.shape

(4800, 9)

In [17]:
is_pos = data.loc[:, 'score'] == 'pos'
df_pos = data.loc[is_pos]

In [18]:
df_pos.head(20)

,Unnamed: 0.1,text,created_at,author_id,id,neg,neu,pos,score
38,38,#28Abr| Desde el @MercantilBanco asistimos a nuestro adultos ¡Para @ivss_oavlp nuestros adultos mayores son importantes! @NicolasMaduro @MagaGutierrezV #IVSS #OAIvss @ivssoficial #AristóbuloPorSiempre https://t.co/UzBid7zSlt,2021-04-28T14:15:59.000Z,2934088562,1387410268939071490,0.017651,0.242634,0.739715,pos
53,53,@MercantilBanco Gracias por su atención la morita,2021-04-27T19:54:22.000Z,139058752,1387133036819582977,0.005218,0.192292,0.802489,pos
80,80,"@MercantilBanco Me parece excelente el haber incluido en Tpago, la modalidad de pago por mensaje de texto, y esta fabuloso el agregar el destinatario a través de un QR nos evita cometer el error al ser escrito por nosotros mismo. Felicitaciones",2021-04-27T07:26:38.000Z,170325456,1386944863573250049,0.003629,0.007116,0.989255,pos
92,92,@BcodeVenezuela Liberen las Divisas Dicom!! Queremos solución @SudebanInforma @BCV_ORG_VE @BcodeVenezuela @la_patilla @ElNacionalWeb @lanacionweb @globovision @descifradocom @elestimulo @DolarToday @ANAUCO @elimpulsocom @TachiraNoticias @TachiraNews @MercantilBanco @mercadosyaccion,2021-04-26T20:04:06.000Z,190265453,1386773099429998594,0.021103,0.189377,0.789520,pos
93,93,@MercantilBanco Gracias...!!!,2021-04-26T19:41:31.000Z,538168349,1386767414004748293,0.004379,0.013057,0.982564,pos
96,96,@MercantilBanco bueno tardes saludos a todos por favor indícame oficina Caracas operativas .\nGracias,2021-04-26T18:50:41.000Z,538168349,1386754622057033729,0.004455,0.294753,0.700792,pos
109,9,@MercantilBanco Agradecida por la informacion feliz semana,2021-04-26T11:35:19.000Z,887348506444353538,1386645060347994112,0.003119,0.013318,0.983563,pos
136,36,"@MercantilBanco Buenos días Estimados en @GTOTALSALUD tenemos a su disposición la Telemedicina, con los mejores profesionales Médicos y especiales en la Medicina para atender su solicitud!!🤳🤳 https://t.co/8x0VwlaoI0",2021-04-25T11:18:15.000Z,1380879351508561920,1386278376894259204,0.007945,0.094775,0.897279,pos
144,44,@MercantilBanco Permitan agregar cuentas.. sería lo máximo,2021-04-24T00:55:20.000Z,1428842382,1385759227621388288,0.008941,0.155534,0.835525,pos
191,91,"No importa donde estés, siempre que hayas sido cliente de @MercantilBanco te deseara feliz cumpleaños https://t.co/IsiElwImsO",2021-04-22T12:05:52.000Z,44057306,1385203197510356995,0.003791,0.040826,0.955383,pos


In [9]:
df_cleaning = data

In [10]:
df_cleaning.dtypes

Unnamed: 0.1      int64
text             object
created_at       object
author_id         int64
id                int64
neg             float64
neu             float64
pos             float64
score            object
dtype: object

## Remover duplicados  mover 1

In [11]:
df_cleaning.drop_duplicates(inplace=True)

In [12]:
df_cleaning.shape

(4798, 9)

## Revisar y Eliminar Null

In [13]:
df_cleaning.isna().sum()

Unnamed: 0.1    0
text            0
created_at      0
author_id       0
id              0
neg             0
neu             0
pos             0
score           0
dtype: int64

In [14]:
df_cleaning.dropna(axis=0, inplace=True)

In [15]:
df_cleaning.isna().sum()

Unnamed: 0.1    0
text            0
created_at      0
author_id       0
id              0
neg             0
neu             0
pos             0
score           0
dtype: int64

## Eliminar menciones

In [16]:
# text = "El @MercantilBanco me quito 180.000 Bs por 4 mensajes de texto al 24024 solicitando clave temporal para un pago móvil, la cual nunca llego, son ladrones o no? https://t.co/bVVmvxfhjH"
# text = " ".join(filter(lambda x:x[0:4]!='http', text.split()))
# text

In [17]:
def remove_link(text):
    text = " ".join(filter(lambda x:x[0:4]!='http', text.split()))
    return text

In [18]:
# df_cleaning['text'] = df_cleaning['text'].apply(remove_link)
# df_cleaning.head()

In [19]:
def remove_mencion(text):
    text = " ".join(filter(lambda x:x[0]!='@', text.split()))
    return text

In [20]:
df_cleaning['text'] = df_cleaning['text'].apply(remove_mencion)
df_cleaning.head()

,Unnamed: 0.1,text,created_at,author_id,id,neg,neu,pos,score
0,0,Buenas tardes intento abrir mi mercantil en línea y dice datos y clave incorrecta...está fallando la plataforma?,2021-04-28T22:47:36.000Z,202148873,1387539021983850497,0.994753,0.004321,0.000926,neg
1,1,Buenas tardes! En esta oportunidad les escribo para saber si hay problemas con la plataforma para realizar transferencias a otras cuentas mercantil ya que al momento de realizar una transferencia me arroja el mensaje de que intente más tardes!,2021-04-28T22:46:07.000Z,1274553941917863936,1387538646832648197,0.666402,0.328037,0.005561,neg
2,2,Deseo es reponer mi tarjeta de débito,2021-04-28T22:32:15.000Z,343710274,1387535156626526209,0.107078,0.886624,0.006298,neu
3,3,"Si lo está, estoy en la misma situación",2021-04-28T22:06:42.000Z,1323258072534552577,1387528726712311812,0.003918,0.989461,0.006621,neu
4,4,"El me quito 180.000 Bs por 4 mensajes de texto al 24024 solicitando clave temporal para un pago móvil, la cual nunca llego, son ladrones o no? https://t.co/bVVmvxfhjH",2021-04-28T21:48:28.000Z,1099360339207049218,1387524137904754689,0.996315,0.002650,0.001035,neg


## Formateo de campo fecha

In [21]:
df_cleaning['created_at'] = pd.to_datetime(df_cleaning['created_at']) #lo hacemos un objeto tiempo
df_cleaning['created_at'] = df_cleaning['created_at'].dt.strftime("%Y-%m-%d") #tomo los elementos que quiero de ese objeto
df_cleaning['created_at'] = pd.to_datetime(df_cleaning['created_at']) #vuelvo a hacerlo un objeto datetime

df_cleaning.head()

,Unnamed: 0.1,text,created_at,author_id,id,neg,neu,pos,score
0,0,Buenas tardes intento abrir mi mercantil en línea y dice datos y clave incorrecta...está fallando la plataforma?,2021-04-28,202148873,1387539021983850497,0.994753,0.004321,0.000926,neg
1,1,Buenas tardes! En esta oportunidad les escribo para saber si hay problemas con la plataforma para realizar transferencias a otras cuentas mercantil ya que al momento de realizar una transferencia me arroja el mensaje de que intente más tardes!,2021-04-28,1274553941917863936,1387538646832648197,0.666402,0.328037,0.005561,neg
2,2,Deseo es reponer mi tarjeta de débito,2021-04-28,343710274,1387535156626526209,0.107078,0.886624,0.006298,neu
3,3,"Si lo está, estoy en la misma situación",2021-04-28,1323258072534552577,1387528726712311812,0.003918,0.989461,0.006621,neu
4,4,"El me quito 180.000 Bs por 4 mensajes de texto al 24024 solicitando clave temporal para un pago móvil, la cual nunca llego, son ladrones o no? https://t.co/bVVmvxfhjH",2021-04-28,1099360339207049218,1387524137904754689,0.996315,0.002650,0.001035,neg


## Remplaza palabra 't pago' por 'tpago'

In [22]:
df_cleaning['text'] = df_cleaning['text'].str.replace('t pago', 'tpago')
df_cleaning.head()

,Unnamed: 0.1,text,created_at,author_id,id,neg,neu,pos,score
0,0,Buenas tardes intento abrir mi mercantil en línea y dice datos y clave incorrecta...está fallando la plataforma?,2021-04-28,202148873,1387539021983850497,0.994753,0.004321,0.000926,neg
1,1,Buenas tardes! En esta oportunidad les escribo para saber si hay problemas con la plataforma para realizar transferencias a otras cuentas mercantil ya que al momento de realizar una transferencia me arroja el mensaje de que intente más tardes!,2021-04-28,1274553941917863936,1387538646832648197,0.666402,0.328037,0.005561,neg
2,2,Deseo es reponer mi tarjeta de débito,2021-04-28,343710274,1387535156626526209,0.107078,0.886624,0.006298,neu
3,3,"Si lo está, estoy en la misma situación",2021-04-28,1323258072534552577,1387528726712311812,0.003918,0.989461,0.006621,neu
4,4,"El me quito 180.000 Bs por 4 mensajes de texto al 24024 solicitando clave temporal para un pago móvil, la cual nunca llego, son ladrones o no? https://t.co/bVVmvxfhjH",2021-04-28,1099360339207049218,1387524137904754689,0.996315,0.002650,0.001035,neg


## Elimina los caracteres iniciales (espacios al principio) y finales (espacios al final)

In [23]:
df_cleaning['text'] = df_cleaning['text'].str.strip()
df_cleaning.head()

,Unnamed: 0.1,text,created_at,author_id,id,neg,neu,pos,score
0,0,Buenas tardes intento abrir mi mercantil en línea y dice datos y clave incorrecta...está fallando la plataforma?,2021-04-28,202148873,1387539021983850497,0.994753,0.004321,0.000926,neg
1,1,Buenas tardes! En esta oportunidad les escribo para saber si hay problemas con la plataforma para realizar transferencias a otras cuentas mercantil ya que al momento de realizar una transferencia me arroja el mensaje de que intente más tardes!,2021-04-28,1274553941917863936,1387538646832648197,0.666402,0.328037,0.005561,neg
2,2,Deseo es reponer mi tarjeta de débito,2021-04-28,343710274,1387535156626526209,0.107078,0.886624,0.006298,neu
3,3,"Si lo está, estoy en la misma situación",2021-04-28,1323258072534552577,1387528726712311812,0.003918,0.989461,0.006621,neu
4,4,"El me quito 180.000 Bs por 4 mensajes de texto al 24024 solicitando clave temporal para un pago móvil, la cual nunca llego, son ladrones o no? https://t.co/bVVmvxfhjH",2021-04-28,1099360339207049218,1387524137904754689,0.996315,0.002650,0.001035,neg


## Se Eliminan los Caracteres Especiales

👇 Remove punctuation and lower case the text.

In [24]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [25]:
mi_puntuacion = '!¡"#$%&\'()*+,-./:;<=>¿?@[\\]^_`{|}~'

In [26]:
def remove_puntuation(text):
    for punctuation in mi_puntuacion:
        text = text.replace(punctuation, " ")
    return text

In [27]:
df_cleaning['text'] = df_cleaning['text'].apply(remove_puntuation)
df_cleaning.head()

,Unnamed: 0.1,text,created_at,author_id,id,neg,neu,pos,score
0,0,Buenas tardes intento abrir mi mercantil en línea y dice datos y clave incorrecta está fallando la plataforma,2021-04-28,202148873,1387539021983850497,0.994753,0.004321,0.000926,neg
1,1,Buenas tardes En esta oportunidad les escribo para saber si hay problemas con la plataforma para realizar transferencias a otras cuentas mercantil ya que al momento de realizar una transferencia me arroja el mensaje de que intente más tardes,2021-04-28,1274553941917863936,1387538646832648197,0.666402,0.328037,0.005561,neg
2,2,Deseo es reponer mi tarjeta de débito,2021-04-28,343710274,1387535156626526209,0.107078,0.886624,0.006298,neu
3,3,Si lo está estoy en la misma situación,2021-04-28,1323258072534552577,1387528726712311812,0.003918,0.989461,0.006621,neu
4,4,El me quito 180 000 Bs por 4 mensajes de texto al 24024 solicitando clave temporal para un pago móvil la cual nunca llego son ladrones o no https t co bVVmvxfhjH,2021-04-28,1099360339207049218,1387524137904754689,0.996315,0.002650,0.001035,neg


## Covierte en minusculas todo

In [28]:
def remove_lower(text1):
    text1 = text1.lower()
    return text1

In [29]:
df_cleaning['text'] = df_cleaning['text'].apply(remove_lower)
df_cleaning.head()

,Unnamed: 0.1,text,created_at,author_id,id,neg,neu,pos,score
0,0,buenas tardes intento abrir mi mercantil en línea y dice datos y clave incorrecta está fallando la plataforma,2021-04-28,202148873,1387539021983850497,0.994753,0.004321,0.000926,neg
1,1,buenas tardes en esta oportunidad les escribo para saber si hay problemas con la plataforma para realizar transferencias a otras cuentas mercantil ya que al momento de realizar una transferencia me arroja el mensaje de que intente más tardes,2021-04-28,1274553941917863936,1387538646832648197,0.666402,0.328037,0.005561,neg
2,2,deseo es reponer mi tarjeta de débito,2021-04-28,343710274,1387535156626526209,0.107078,0.886624,0.006298,neu
3,3,si lo está estoy en la misma situación,2021-04-28,1323258072534552577,1387528726712311812,0.003918,0.989461,0.006621,neu
4,4,el me quito 180 000 bs por 4 mensajes de texto al 24024 solicitando clave temporal para un pago móvil la cual nunca llego son ladrones o no https t co bvvmvxfhjh,2021-04-28,1099360339207049218,1387524137904754689,0.996315,0.002650,0.001035,neg


## Elimina Acentos

In [30]:
from unicodedata import normalize
def cambia_acentuadas(text):
    trans_tab = dict.fromkeys(map(ord, u'\u0301\u0308'), None)
    text = normalize('NFKC', normalize('NFKD', text).translate(trans_tab))
    return text

In [31]:
df_cleaning['text'] = df_cleaning['text'].apply(cambia_acentuadas)
df_cleaning.head()

,Unnamed: 0.1,text,created_at,author_id,id,neg,neu,pos,score
0,0,buenas tardes intento abrir mi mercantil en linea y dice datos y clave incorrecta esta fallando la plataforma,2021-04-28,202148873,1387539021983850497,0.994753,0.004321,0.000926,neg
1,1,buenas tardes en esta oportunidad les escribo para saber si hay problemas con la plataforma para realizar transferencias a otras cuentas mercantil ya que al momento de realizar una transferencia me arroja el mensaje de que intente mas tardes,2021-04-28,1274553941917863936,1387538646832648197,0.666402,0.328037,0.005561,neg
2,2,deseo es reponer mi tarjeta de debito,2021-04-28,343710274,1387535156626526209,0.107078,0.886624,0.006298,neu
3,3,si lo esta estoy en la misma situacion,2021-04-28,1323258072534552577,1387528726712311812,0.003918,0.989461,0.006621,neu
4,4,el me quito 180 000 bs por 4 mensajes de texto al 24024 solicitando clave temporal para un pago movil la cual nunca llego son ladrones o no https t co bvvmvxfhjh,2021-04-28,1099360339207049218,1387524137904754689,0.996315,0.002650,0.001035,neg


## Remueve Numeros

In [32]:
def remove_numbers(text1):
    text1 = ''.join(word for word in text1 if not word.isdigit())
    return text1

In [33]:
df_cleaning['text'] = df_cleaning['text'].apply(remove_numbers)
df_cleaning.head()

,Unnamed: 0.1,text,created_at,author_id,id,neg,neu,pos,score
0,0,buenas tardes intento abrir mi mercantil en linea y dice datos y clave incorrecta esta fallando la plataforma,2021-04-28,202148873,1387539021983850497,0.994753,0.004321,0.000926,neg
1,1,buenas tardes en esta oportunidad les escribo para saber si hay problemas con la plataforma para realizar transferencias a otras cuentas mercantil ya que al momento de realizar una transferencia me arroja el mensaje de que intente mas tardes,2021-04-28,1274553941917863936,1387538646832648197,0.666402,0.328037,0.005561,neg
2,2,deseo es reponer mi tarjeta de debito,2021-04-28,343710274,1387535156626526209,0.107078,0.886624,0.006298,neu
3,3,si lo esta estoy en la misma situacion,2021-04-28,1323258072534552577,1387528726712311812,0.003918,0.989461,0.006621,neu
4,4,el me quito bs por mensajes de texto al solicitando clave temporal para un pago movil la cual nunca llego son ladrones o no https t co bvvmvxfhjh,2021-04-28,1099360339207049218,1387524137904754689,0.996315,0.002650,0.001035,neg


## Separar palabras

In [34]:
def remove_StopWords(text):
    spanish_stopwords = set(stopwords.words('spanish'))
    #stop_words = set(stopwords.words('spanish'))
    word_tokens = word_tokenize(text) 
    text = [w for w in word_tokens if not w in spanish_stopwords]
    return text

In [35]:
df_cleaning['text'] = df_cleaning['text'].apply(remove_StopWords)
df_cleaning.head()

,Unnamed: 0.1,text,created_at,author_id,id,neg,neu,pos,score
0,0,"[buenas, tardes, intento, abrir, mercantil, linea, dice, datos, clave, incorrecta, fallando, plataforma]",2021-04-28,202148873,1387539021983850497,0.994753,0.004321,0.000926,neg
1,1,"[buenas, tardes, oportunidad, escribo, saber, si, problemas, plataforma, realizar, transferencias, cuentas, mercantil, momento, realizar, transferencia, arroja, mensaje, intente, mas, tardes]",2021-04-28,1274553941917863936,1387538646832648197,0.666402,0.328037,0.005561,neg
2,2,"[deseo, reponer, tarjeta, debito]",2021-04-28,343710274,1387535156626526209,0.107078,0.886624,0.006298,neu
3,3,"[si, misma, situacion]",2021-04-28,1323258072534552577,1387528726712311812,0.003918,0.989461,0.006621,neu
4,4,"[quito, bs, mensajes, texto, solicitando, clave, temporal, pago, movil, nunca, llego, ladrones, https, t, co, bvvmvxfhjh]",2021-04-28,1099360339207049218,1387524137904754689,0.996315,0.002650,0.001035,neg


## Lemmatizer

In [36]:
from nltk.stem import WordNetLemmatizer
def remove_Lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(word) for word in text]
    str_lemmatizer = " ".join(lemmatized)
    return str_lemmatizer

In [37]:
df_cleaning['text'] = df_cleaning['text'].apply(remove_Lemmatize)
df_cleaning.head()

,Unnamed: 0.1,text,created_at,author_id,id,neg,neu,pos,score
0,0,buenas tardes intento abrir mercantil linea dice datos clave incorrecta fallando plataforma,2021-04-28,202148873,1387539021983850497,0.994753,0.004321,0.000926,neg
1,1,buenas tardes oportunidad escribo saber si problemas plataforma realizar transferencias cuentas mercantil momento realizar transferencia arroja mensaje intente ma tardes,2021-04-28,1274553941917863936,1387538646832648197,0.666402,0.328037,0.005561,neg
2,2,deseo reponer tarjeta debito,2021-04-28,343710274,1387535156626526209,0.107078,0.886624,0.006298,neu
3,3,si misma situacion,2021-04-28,1323258072534552577,1387528726712311812,0.003918,0.989461,0.006621,neu
4,4,quito b mensajes texto solicitando clave temporal pago movil nunca llego ladrones http t co bvvmvxfhjh,2021-04-28,1099360339207049218,1387524137904754689,0.996315,0.002650,0.001035,neg


In [38]:
df_cleaning['text']

0                                                                                                                                              buenas tardes intento abrir mercantil linea dice datos clave incorrecta fallando plataforma
1                                                                buenas tardes oportunidad escribo saber si problemas plataforma realizar transferencias cuentas mercantil momento realizar transferencia arroja mensaje intente ma tardes
2                                                                                                                                                                                                             deseo reponer tarjeta debito
3                                                                                                                                                                                                                       si misma situacion
4                                                           

In [39]:
df_cleaning.shape

(4798, 9)

## Definir Train y test

In [40]:
X = df_cleaning['text']
y = df_cleaning['score']

In [41]:

# fig, ax = plt.subplots()
# #Colocamos una etiqueta en el eje Y
# ax.set_ylabel('Twitts')
# #Colocamos una etiqueta en el eje X
# ax.set_title('Cantidad de Twitts')
# #Creamos la grafica de barras utilizando 'paises' como eje X y 'ventas' como eje y.
# plt.bar(X, y)
# plt.savefig('Fecha.png')
# #Finalmente mostramos la grafica con el metodo show()
# plt.show()


In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [43]:
X_train

705                         buenos dia saludos problema punto bloqueo tarjeta trato ingresar cambiar clave dice nro tarjeta invalido llamo nro puedo comunicarme debo hacer
339                                                                                                                     buenos dia mercantil empresa puede hacer pago movil
661               pregunto van cobrar factura servicio años reclamo averia cantv envio cuadrilla señores pretenden cobrar dolares reparacion pregunto serum fabrico dolares
609                                                                                                                                                        ustedes sola app
626                                                                   buenas tardes si deseo vender moneda extranjera ustedes seria proceso cuanto tiempo disponible cuenta
                                                                                       ...                                                  

### Pipeline modelo

In [44]:
# Create Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
#from sklearn.linear_model import LogisticRegression
from scipy import stats
import warnings

warnings.filterwarnings('ignore')

# Create Pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(lowercase = False)),
    ('model', MultinomialNB()),
])

# Set parameters to search
#parameters = {
 #   'tfidf__ngram_range': [(1,1), (2,2), (1,2)],
  #  'tfidf__min_df': stats.loguniform(0.01, .5),
   # 'tfidf__max_df': stats.loguniform(0.01, .5),
    #'model__alpha': stats.loguniform(0.01, 10),
#}

parameters = {
    'tfidf__ngram_range': [(1,1), (2,2), (1,2),],
    'tfidf__min_df': stats.loguniform(0.01, 0.2),
    'tfidf__max_df': stats.loguniform(0.01, 0.3),
    'tfidf__norm' : ['l1', 'l2'] ,
    'model__alpha': stats.uniform(0.5, 1)
}

# Perform grid search on pipeline
grid_search = RandomizedSearchCV(pipeline, parameters, n_jobs=-1, 
                           verbose=1, scoring = "accuracy", cv=5, n_iter=200, refit=True)

grid_search.fit(X_train,y_train)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('tfidf',
                                              TfidfVectorizer(lowercase=False)),
                                             ('model', MultinomialNB())]),
                   n_iter=200, n_jobs=-1,
                   param_distributions={'model__alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f091c8007c0>,
                                        'tfidf__max_df': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f091c800610>,
                                        'tfidf__min_df': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f091c446070>,
                                        'tfidf__ngram_range': [(1, 1), (2, 2),
                                                               (1, 2)],
                                        'tfidf__norm': ['l1', 'l2']},
                   scoring='accuracy', verbose=1)

##RandomizedSearchCV

In [45]:
grid_search.best_score_

0.6503845539706197

In [46]:
grid_search.best_params_

{'model__alpha': 1.2727165570087884,
 'tfidf__max_df': 0.21628562858285544,
 'tfidf__min_df': 0.012254343974953454,
 'tfidf__ngram_range': (1, 1),
 'tfidf__norm': 'l2'}

In [47]:
grid_search.best_estimator_

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(lowercase=False, max_df=0.21628562858285544,
                                 min_df=0.012254343974953454)),
                ('model', MultinomialNB(alpha=1.2727165570087884))])

In [48]:
grid_search.best_estimator_.score(X_test, y_test)

0.6777777777777778

In [49]:
result = grid_search.best_estimator_.predict(X_test)

In [50]:
result_df = pd.DataFrame(result)

In [51]:
result_df[0].unique()

array(['neg', 'neu', 'pos'], dtype=object)

In [52]:
X_test.head(2)

63                                                                                                           realice transferencia cuenta banco hace dia aun aparece
430    buenas tardes puede desbloquear tarjeta debito paguina web aplicacion atencion cliente da opciones misma necesitamos respuestas urgencia aca intentando mismo
Name: text, dtype: object

In [53]:
X_test.head()

63                                                                                                           realice transferencia cuenta banco hace dia aun aparece
430    buenas tardes puede desbloquear tarjeta debito paguina web aplicacion atencion cliente da opciones misma necesitamos respuestas urgencia aca intentando mismo
687                                                                                                                    amigo puede hoy active pagaste dia cuarentena
389                                                malditos coños madre casa vamos pagar ma factura arreglen linea me pagando servicio q estan dando siento estafado
227                      sres hace dia intento hacer reclamo transaccion reconocida telefono dicen pagina web pagina web dice agencia finalmente procedimiento favor
Name: text, dtype: object

In [54]:
grid_search.cv_results_

{'mean_fit_time': array([0.13202744, 0.12808204, 0.07952805, 0.08987999, 0.05391278,
        0.12505493, 0.04629726, 0.11414852, 0.08696713, 0.08715491,
        0.09364705, 0.06843176, 0.09367146, 0.09525371, 0.08809271,
        0.08930535, 0.06949019, 0.09568052, 0.07634521, 0.06286149,
        0.17763982, 0.06487427, 0.08835173, 0.07891045, 0.08625641,
        0.14460707, 0.0964581 , 0.11779714, 0.06124964, 0.05234208,
        0.0639194 , 0.11638412, 0.12152581, 0.12730265, 0.05967212,
        0.05786166, 0.1313993 , 0.10840378, 0.05998917, 0.1178587 ,
        0.07012997, 0.12495227, 0.13527012, 0.08749928, 0.05600615,
        0.11616988, 0.08177776, 0.06820421, 0.07853212, 0.12070317,
        0.07333302, 0.12764149, 0.13955712, 0.10282102, 0.16729364,
        0.10421638, 0.13364139, 0.09216242, 0.11589069, 0.06296215,
        0.06692829, 0.06506095, 0.0671205 , 0.11266994, 0.08798876,
        0.06909552, 0.0632381 , 0.06812897, 0.14196825, 0.09693775,
        0.0937242 , 0.15751486,

In [55]:
for i in X_test.keys():
    result[i] = X_test[i]

ValueError: setting an array element with a sequence

### Learning Curves

In [ ]:
#from sklearn.linear_model import LinearRegression
# se crea la lista de verificacion
train_sizes = [25,50,75,100,250,500,750,1000,1150]
# se crea la linea de regresion
train_sizes, train_scores, test_scores = learning_curve(
    estimator=grid_search.best_estimator_, X=X_train, y=y_train, train_sizes=train_sizes, cv=5)
# Se calcula la media de los score de cada una de los cv ejecutados
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
	
# Se grafica los score
plt.plot(train_sizes, train_scores_mean, label = 'Training score')
plt.plot(train_sizes, test_scores_mean, label = 'Test score')
plt.ylabel('r2 score', fontsize = 14)
plt.xlabel('Training set size', fontsize = 14)
plt.title('Learning curves', fontsize = 18, y = 1.03)
plt.legend()

In [ ]:
result[['neg','neu','pos']].plot(kind='box',figsize = (10,7))